<a href="https://colab.research.google.com/github/AffanA2003/X-ray-Dental-Detection/blob/main/final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import json
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



In [ ]:
csv_path = 'dataset.csv'
df = pd.read_csv(csv_path)



In [ ]:
img_dir = 'images/'


In [ ]:
image_paths = []
annotations = []
labels = []
id = []

for index, row in df.iterrows():
    img_filename = row['filename']
    
    # Parse the string values into dictionaries
    region_shape_attributes = json.loads(row['region_shape_attributes'])
    region_attributes = json.loads(row['region_attributes'])
    id = df['region_id'].values
    
    annotation = {
        'x': region_shape_attributes['x'],
        'y': region_shape_attributes['y'],
        'width': region_shape_attributes['width'],
        'height': region_shape_attributes['height']
    }
    status = region_attributes['status']
    
    
    img_path = os.path.join(img_dir, img_filename)
    
    image_paths.append(img_path)
    annotations.append(annotation)
    labels.append(status)




In [ ]:
annotations = np.array(annotations)
ids = np.array(id)
labels = np.array(labels)
image_paths, annotations, labels, ids = shuffle(image_paths, annotations, labels, ids, random_state=42)
train_image_paths, test_image_paths, train_annotations, test_annotations, train_labels, test_labels, train_ids, test_ids = train_test_split(
    image_paths, annotations, labels, ids, test_size=0.2, random_state=42)


In [ ]:
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img = np.array(img.convert('RGB')) / 255.0  # Convert to RGB and normalize
    return img

train_annotations = np.array(train_annotations)
test_annotations = np.array(test_annotations)
train_images = np.array([load_and_preprocess_image(img_path) for img_path in train_image_paths])
test_images = np.array([load_and_preprocess_image(img_path) for img_path in test_image_paths])
# Rest of your code...


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)




In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_split=0.2)



In [ ]:
model.save('keras_model.h5')

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

# Make predictions on the test data
test_predictions = model.predict(test_images)

# Convert the predicted labels to class labels
test_predicted_labels = np.argmax(test_predictions, axis=1)

# Calculate the F1 score
from sklearn.metrics import f1_score

f1 = f1_score(test_labels, test_predicted_labels, average='macro')

# Print the evaluation results
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("F1 Score:", f1)
